
In this notebook, You will do amazon review classification with BERT.

In [ ]:
#all imports
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model

import re
import pickle
from sklearn.model_selection import train_test_split

import random as rn

from tensorflow.keras.layers import Input, Dense, Activation, Dropout,MaxPooling1D,Conv1D
#from tensorflow.keras.models import Model
import datetime
from sklearn.metrics import roc_auc_score


In [ ]:
tf.__version__

'2.8.2'

In [ ]:
tf.test.is_built_with_gpu_support()

True

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

<font size=4>Grader function 1 </font>

In [ ]:
def grader_tf_version():
    assert((tf.__version__)>'2')
    return True
grader_tf_version()

True

<pre><font size=6>Part-1: Preprocessing</font></pre>

In [ ]:
#Read the dataset - Amazon fine food reviews
#reviews = pd.read_csv(r"D:\ML\Internal DL\NLP\amazon-fine-food-reviews\Reviews.csv")
reviews_df = pd.read_csv(r"/content/26_NLP_Transfer/Reviews.csv")
#check the info of the dataset
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [ ]:
reviews_df.shape

(568454, 10)

In [ ]:
reviews_df.shape

(568454, 10)

In [ ]:
reviews_df.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


In [ ]:
#get only 2 columns - Text, Score
#drop the NAN values

reviews = reviews_df.loc[ :, ['Text','Score']]
reviews = reviews.dropna(inplace=False)
#reviews = reviews.reset_index(drop=True)
reviews.shape   #.dropna(inplace=False).shape

(568454, 2)

In [ ]:
#if score> 3, set score = 1
#if score<=2, set score = 0
#if score == 3, remove the rows. 

#443766 # 82007
reviews = reviews[ reviews['Score'] !=3 ]
print( reviews.shape )

(525814, 2)


In [ ]:
def set_binary_score( score ):
  if score > 3 :
    return 1
  elif score <= 2 :
    return 0
  else :
    return score

#reviews['Score'] = reviews['Score'].apply( set_binary_score  )
reviews.shape

(525814, 2)

In [ ]:
print( reviews[ reviews['Score'] ==0 ].count() )
print( reviews[ reviews['Score'] ==1 ].count() )
#reviews.Score.value_counts()

Text     82037
Score    82037
dtype: int64
Text     443777
Score    443777
dtype: int64


<font size=4>Grader function 2 </font>

In [ ]:
def grader_reviews():
    temp_shape = (reviews.shape == (525814, 2)) and (reviews.Score.value_counts()[1]==443777)
    assert(temp_shape == True)
    return True
grader_reviews()

True

In [ ]:
def get_wordlen(x):
    return len(x.split())
reviews['len'] = reviews.Text.apply(get_wordlen)
reviews = reviews[reviews.len<50]
reviews = reviews.sample(n=100000, random_state=30)

In [ ]:
#remove HTML from the Text column and save in the Text column only

In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def text_preprocess( text_line ):
    processed_text = text_line
    
    processed_text = processed_text.replace('\r', ' ')
    processed_text = processed_text.replace('\n', ' ')
    processed_text = processed_text.replace('\t', ' ')
    processed_text = processed_text.replace('\"', ' ')
    
    processed_text = decontracted( processed_text )

    remove_tag_regex = re.compile(r'<[^>]+>')
    processed_text = remove_tag_regex.sub( ' ' , processed_text )

    sp_char_regex = re.compile(r'[^a-zA-Z0-9]')
    processed_text = sp_char_regex.sub( ' ' , processed_text )
    processed_text = processed_text.lower()

    return processed_text

#reviews['Text'] = reviews['Text'].apply( text_preprocess )

In [ ]:
reviews.shape

(100000, 3)

In [ ]:
reviews['Score'].value_counts()

1    87004
0    12996
Name: Score, dtype: int64

In [ ]:
#print head 5
reviews.head()

,Text,Score,len
64117,The tea was of great quality and it tasted lik...,1,30
418112,My cat loves this. The pellets are nice and s...,1,31
357829,Great product. Does not completely get rid of ...,1,41
175872,This gum is my favorite! I would advise every...,1,27
178716,I also found out about this product because of...,1,22


In [ ]:
reviews[3:4].values

array([['these bars are so yummy  i was looking for a night snack and grabbed this out of my mom box from influenster and this hit the spot ',
        1, 27],
       ['my vodka tonics have never been so refreshing   i wish i would have ordered two cases now as my husband and i are going through far too much ',
        1, 28],
       ['i got the pineapple  mango and choco  my family liked the latter 2 the most  yummy coconut water if you are not looking for one that is not too sugary imo ',
        1, 31],
       ['when i bought the coffee maker  this was in the free samples   after using the samples my husband liked the donut coffee the best and so did i  i buy both the decaf and the regular k cup coffee people donut shop   love it ',
        1, 43],
       ['we use this on fish also  it is a great seasoning and is a low cal solution to a great meal ',
        1, 21],
       ['is a good product and taste much better than the small squares that are sold under another name   good candy',

In [ ]:
reviews[30:40].values

array([["I have never been a huge coffee fan. However, my mother purchased this little machine and talked me into trying the Latte Macciato. No Coffee Shop has a better one and I like most of the other products, too (as a usually non-coffee drinker!). The little Dolche Guesto Machine is super easy to use and prepares a really good Coffee/Latte/Cappuccino/etc in less than a minute (if water is heated up). I would recommend the Dolce Gusto to anyone. Too good for the price and I'am getting one myself! :)",
        1],
       ['This offer is a great price and a great taste, thanks Amazon for selling this product.  Staral',
        1],
       ["McCann's Instant Oatmeal is great if you must have your oatmeal but can only scrape together two or three minutes to prepare it. There is no escaping the fact, however, that even the best instant oatmeal is nowhere near as good as even a store brand of oatmeal requiring stovetop preparation.  Still, the McCann's is as good as it gets for instant oat

In [ ]:
#import pickle
#from sklearn.model_selection import train_test_split

In [ ]:
#split the data into train and test data(20%) with Stratify sampling, random state 33, 

X_train , X_test , y_train , y_test = train_test_split( reviews['Text'].values ,
                                                       reviews['Score'].values ,
                                                       test_size=0.20, stratify= reviews['Score'].values , random_state=33 )

X_train.shape , X_test.shape , y_train.shape , y_test.shape

((80000,), (20000,), (80000,), (20000,))

In [ ]:
X_train[0:1]

array(['i had never tried this brand before  so i was worried about the quality   it tasted great   a very nice smooth rich full flavor   its my new favoret '],
      dtype=object)

In [ ]:
#plot bar graphs of y_train and y_test

In [ ]:
#saving to disk. if we need, we can load preprocessed data directly. 
#reviews.to_csv( dir_path + 'preprocessed_1.csv', index=False)


<pre><font size=6>Part-2: Creating BERT Model</font> 

If you want to know more about BERT, You can watch live sessions on Transformers and BERt. 
we will strongly recommend you to read <a href="https://jalammar.github.io/illustrated-transformer/">Transformers</a>, <a href="https://arxiv.org/abs/1810.04805">BERT Paper</a> and, <a href="https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/">This blog</a>.


For this assignment, we are using <a href="https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1">BERT uncased Base model</a>. 
It uses L=12 hidden layers (i.e., Transformer blocks), a hidden size of H=768, and A=12 attention heads. </pre>

In [ ]:
## Loading the Pretrained Model from tensorflow HUB
tf.keras.backend.clear_session()

# maximum length of a seq in the data we have, for now i am making it as 55. You can change this
max_seq_length = 55

#BERT takes 3 inputs

#this is input words. Sequence of words represented as integers
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")

#mask vector if you are padding anything
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")

#segment vectors. If you are giving only one sentence for the classification, total seg vector is 0. 
#If you are giving two sentenced with [sep] token separated, first seq segment vectors are zeros and 
#second seq segment vector are 1's
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")

#bert layer 
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

#Bert model
#We are using only pooled output not sequence out. 
#If you want to know about those, please read https://www.kaggle.com/questions-and-answers/86510
bert_model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=pooled_output)


In [ ]:
bert_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 55)]         0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 55)]         0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 55)]         0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 55, 768)]                 'input_mask[0][0]',         

In [ ]:
#bert_model.get_layer('keras_layer').layer()
#bert_model.layers[3]._layers
#bert_model.layers[0].layers()


In [ ]:
#bert_layer.summary()

In [ ]:
bert_model.output

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'keras_layer')>

In [ ]:
'''
def build_classifier_model_1():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)
'''


<pre><font size=6>Part-3: Tokenization</font></pre>

In [ ]:
#getting Vocab file
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [ ]:
#import tokenization - We have given tokenization.py file

In [ ]:
'''
import pickle
'''
import sys
sys.path.append('/content/26_NLP_Transfer')
from tokenization import FullTokenizer


In [ ]:
#from tokenization import FullTokenizer

In [ ]:
# Create tokenizer " Instantiate FullTokenizer" 
# name must be "tokenizer"
# the FullTokenizer takes two parameters 1. vocab_file and 2. do_lower_case 
# we have created these in the above cell ex: FullTokenizer(vocab_file, do_lower_case )
# please check the "tokenization.py" file the complete implementation

In [ ]:
tokenizer = FullTokenizer(vocab_file, do_lower_case )

<font size=4>Grader function 3 </font>

In [ ]:
#it has to give no error 
def grader_tokenize(tokenizer):
    out = False
    try:
        out=('[CLS]' in tokenizer.vocab) and ('[SEP]' in tokenizer.vocab)
    except:
        out = False
    assert(out==True)
    return out
grader_tokenize(tokenizer)

True

In [ ]:
# Create train and test tokens (X_train_tokens, X_test_tokens) from (X_train, X_test) using Tokenizer and 

# add '[CLS]' at start of the Tokens and '[SEP]' at the end of the tokens. 

# maximum number of tokens is 55(We already given this to BERT layer above) so shape is (None, 55)

# if it is less than 55, add '[PAD]' token else truncate the tokens length.(similar to padding)

# Based on padding, create the mask for Train and Test ( 1 for real token, 0 for '[PAD]'), 
# it will also same shape as input tokens (None, 55) save those in X_train_mask, X_test_mask

# Create a segment input for train and test. We are using only one sentence so all zeros. This shape will also (None, 55)

# type of all the above arrays should be numpy arrays

# after execution of this cell, you have to get 
# X_train_tokens, X_train_mask, X_train_segment
# X_test_tokens, X_test_mask, X_test_segment

In [ ]:
#reviews['Text'].tolist()
#X_train_tokens , X_train_mask, X_train_segment = [] , [] , []
#X_train_tokens[1]

In [ ]:

X_train_tokens , X_train_mask, X_train_segment = [] , [] , []

for i in range(X_train.shape[0]):
    
    tokens = tokenizer.tokenize( ( X_train[i] ) )
    tokens = tokens[ 0 : ( max_seq_length - 2 )]
    tokens = ['[CLS]',*tokens,'[SEP]']

    #word to integer
    X_train_tokens.append( np.array( tokenizer.convert_tokens_to_ids(tokens) +  [0]*( max_seq_length - len(tokens))  ) )
    
    # mask
    X_train_mask.append( np.array([1]*len(tokens) + [0]*( max_seq_length - len(tokens))))
    
    #segment
    X_train_segment.append(np.array( [0]*( max_seq_length )))

X_train_tokens = np.array( X_train_tokens )
X_train_mask = np.array( X_train_mask )
X_train_segment = np.array( X_train_segment ) 


In [ ]:
#np.array( X_train ).tolist()

X_test_tokens, X_test_mask, X_test_segment = [] , [] , []

for i in range(X_test.shape[0]):
    
    tokens = tokenizer.tokenize( ( X_test[i] ) )
    tokens = tokens[ 0 : ( max_seq_length - 2 )]
    tokens = ['[CLS]',*tokens,'[SEP]']
    
    X_test_tokens.append( np.array( tokenizer.convert_tokens_to_ids(tokens) +  [0]*( max_seq_length - len(tokens))  ) )
    
    X_test_mask.append( np.array([1]*len(tokens) + [0]*( max_seq_length - len(tokens))))
    
    X_test_segment.append(np.array( [0]*( max_seq_length )))

X_test_tokens = np.array( X_test_tokens )
X_test_mask = np.array( X_test_mask )
X_test_segment = np.array( X_test_segment )


In [ ]:
X_train_tokens.shape , X_train_mask.shape , X_train_segment.shape , \
X_test_tokens.shape , X_test_mask.shape , X_test_segment.shape

((80000, 55), (80000, 55), (80000, 55), (20000, 55), (20000, 55), (20000, 55))

In [ ]:
X_train_tokens[0:1] , X_train_mask[0:1] , X_train_segment[0:1]


(array([[  101,  1045,  2018,  2196,  2699,  2023,  4435,  2077,  2061,
          1045,  2001,  5191,  2055,  1996,  3737,  2009, 12595,  2307,
          1037,  2200,  3835,  5744,  4138,  2440, 14894,  2049,  2026,
          2047,  5684,  3388,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]]),
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

#### Example
<img src='https://i.imgur.com/5AhhmgU.png'>

In [ ]:
#import pickle

In [ ]:
data_path = '/content/data/'

In [ ]:
##save all your results to disk so that, no need to run all again. 
#pickle.dump((X_train, X_train_tokens, X_train_mask, X_train_segment, y_train),open( data_path + 'train_data_1.pkl','wb'))
#pickle.dump((X_test, X_test_tokens, X_test_mask, X_test_segment, y_test),open( data_path + 'test_data_1.pkl','wb'))


In [ ]:
#you can load from disk
X_train, X_train_tokens, X_train_mask, X_train_segment, y_train = pickle.load(open( data_path + "train_data_1.pkl", 'rb')) 
X_test, X_test_tokens, X_test_mask, X_test_segment, y_test = pickle.load(open( data_path + "test_data_1.pkl", 'rb'))


In [ ]:
#max_seq_length = 55
X_train_tokens.shape

(80000, 55)

<font size=4>Grader function 4 </font>

In [ ]:
def grader_alltokens_train():
    out = False
    
    if type(X_train_tokens) == np.ndarray:
        
        temp_shapes = (X_train_tokens.shape[1]==max_seq_length) and (X_train_mask.shape[1]==max_seq_length) and \
        (X_train_segment.shape[1]==max_seq_length)
        
        segment_temp = not np.any(X_train_segment)
        
        mask_temp = np.sum(X_train_mask==0) == np.sum(X_train_tokens==0)
        
        no_cls = np.sum(X_train_tokens==tokenizer.vocab['[CLS]'])==X_train_tokens.shape[0]
        
        no_sep = np.sum(X_train_tokens==tokenizer.vocab['[SEP]'])==X_train_tokens.shape[0]
        
        out = temp_shapes and segment_temp and mask_temp and no_cls and no_sep
      
    else:
        print('Type of all above token arrays should be numpy array not list')
        out = False
    assert(out==True)
    return out

grader_alltokens_train()

True

<font size=4>Grader function 5 </font>

In [ ]:
def grader_alltokens_test():
    out = False
    if type(X_test_tokens) == np.ndarray:
        
        temp_shapes = (X_test_tokens.shape[1]==max_seq_length) and (X_test_mask.shape[1]==max_seq_length) and \
        (X_test_segment.shape[1]==max_seq_length)
        
        segment_temp = not np.any(X_test_segment)
        
        mask_temp = np.sum(X_test_mask==0) == np.sum(X_test_tokens==0)
        
        no_cls = np.sum(X_test_tokens==tokenizer.vocab['[CLS]'])==X_test_tokens.shape[0]
        
        no_sep = np.sum(X_test_tokens==tokenizer.vocab['[SEP]'])==X_test_tokens.shape[0]
        
        out = temp_shapes and segment_temp and mask_temp and no_cls and no_sep
      
    else:
        print('Type of all above token arrays should be numpy array not list')
        out = False
    assert(out==True)
    return out
grader_alltokens_test()

True

<pre><font size=6>Part-4: Getting Embeddings from BERT Model</font>
We already created the BERT model in the part-2 and input data in the part-3. 
We will utlize those two and will get the embeddings for each sentence in the 
Train and test data.</pre>

In [ ]:
bert_model.input

[<KerasTensor: shape=(None, 55) dtype=int32 (created by layer 'input_word_ids')>,
 <KerasTensor: shape=(None, 55) dtype=int32 (created by layer 'input_mask')>,
 <KerasTensor: shape=(None, 55) dtype=int32 (created by layer 'segment_ids')>]

In [ ]:
bert_model.output

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'keras_layer')>

In [ ]:
#import random as rn
'''
def aucroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)
'''

In [ ]:
# this function not working

def build_classifier_model():
  tf.keras.backend.clear_session()
  np.random.seed(0)
  rn.seed(0)

  #text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  #preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  #encoder_inputs = preprocessing_layer(text_input)
  #encoder_inputs = []
  
  #this is input word. Sequence of words represented as integers
  max_seq_length = 55
  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")
  segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")

  tfhub_handle_encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=False , name='BERT_encoder')
  outputs = encoder([ input_word_ids , input_mask , segment_ids ])

  net1 = outputs['pooled_output']
  net2 = tf.keras.layers.Dropout(0.1)(net1)
  #net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  #dense4 = Dense(units=32,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=32),name='dense4')(dropout_1)
  #dropout_2 = Dropout(0.1)(dense4)

  out1 = Dense(units=1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=34),name='output')(net2)

  temp_model = tf.keras.Model( [input_word_ids, input_mask, segment_ids] , out1 )
  #temp_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy',aucroc])
  
  return temp_model

#temp_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy',aucroc])


In [ ]:
# this function works
def build_bert_model_2( ):
    tf.keras.backend.clear_session()
    np.random.seed(0)
    rn.seed(0)

    max_len=55
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False ,name='BERT_encoder')
    pooled_output , sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    print( pooled_output )
    
    #clf_output = None #sequence_output[:, 0, :]
    
    net = tf.keras.layers.Dense(64, activation='relu')( pooled_output )
    net = tf.keras.layers.Dropout(0.1)(net)
    #net = tf.keras.layers.Dense(16, activation='relu')(net)
    #net = tf.keras.layers.Dropout(0.1)(net)
    out = tf.keras.layers.Dense(1, activation='sigmoid')(net)
    
    model2 = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)

    model2.compile(tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy', aucroc ])
    
    return model2


In [ ]:
bert_model_1 = build_bert_model_2()

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='BERT_encoder/StatefulPartitionedCall:0', description="created by layer 'BERT_encoder'")


In [ ]:
bert_model_1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 55)]         0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 55)]         0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 55)]         0           []                               
                                                                                                  
 BERT_encoder (KerasLayer)      [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 55, 768)]                 'input_mask[0][0]',         

In [ ]:

bert_model_1.fit(x = [X_train_tokens, X_train_mask, X_train_segment ] , y=y_train , batch_size=256 , epochs=1 ) #, \
#          validation_data=([X_test_tokens, X_test_mask, X_test_segment] , y_test ) , validation_batch_size=256  )


313/313 [==============================] - 305s 926ms/step - loss: 0.4292 - accuracy: 0.8467 - aucroc: 0.5776


In [ ]:
#bert_model_1.fit()

bert_model_1.fit(x = [X_train_tokens, X_train_mask, X_train_segment] , y=y_train , batch_size=256 , epochs=10 , \
          validation_data=([X_test_tokens, X_test_mask, X_test_segment] , y_test ) , validation_batch_size=256  )


Epoch 1/10
313/313 [==============================] - 388s 1s/step - loss: 0.0000e+00 - accuracy: 0.1355 - val_loss: 0.0000e+00 - val_accuracy: 0.1300
Epoch 2/10
 17/313 [>.............................] - ETA: 4:42 - loss: 0.0000e+00 - accuracy: 0.1252

KeyboardInterrupt: ignored

In [ ]:
# get the train output, BERT model will give one output so save in
# X_train_pooled_output
#X_train_pooled_output=bert_model.predict([X_train_tokens, X_train_mask, X_train_segment])

#X_train_pooled_output=bert_model([X_train_tokens, X_train_mask, X_train_segment])

In [ ]:
# get the train output, BERT model will give one output so save in
# X_train_pooled_output
X_train_pooled_output=bert_model.predict([X_train_tokens, X_train_mask, X_train_segment])


In [ ]:
X_train_pooled_output.shape

(80000, 768)

In [ ]:
# get the test output, BERT model will give one output so save in
# X_test_pooled_output
X_test_pooled_output = bert_model.predict([X_test_tokens, X_test_mask, X_test_segment])

#X_test_pooled_output = bert_model([X_test_tokens, X_test_mask, X_test_segment])


In [ ]:
X_test_pooled_output.shape

(20000, 768)

In [ ]:
##save all your results to disk so that, no need to run all again. 
#pickle.dump((X_train_pooled_output, X_test_pooled_output),open( data_path + 'final_output_1.pkl','wb'))

In [ ]:
#X_train_pooled_output, X_test_pooled_output= pickle.load(open('final_output_1.pkl', 'rb'))

In [ ]:
#X_train_pooled_output, X_test_pooled_output= pickle.load(open( data_path + 'final_output_1.pkl', 'rb'))

In [ ]:
X_train_pooled_output.shape , X_test_pooled_output.shape


((80000, 768), (20000, 768))

<font size=4>Grader function 6 </font>

In [ ]:
#now we have X_train_pooled_output, y_train
#X_test_pooled_ouput, y_test

#please use this grader to evaluate
def greader_output():
    assert(X_train_pooled_output.shape[1]==768)
    assert(len(y_train)==len(X_train_pooled_output))
    assert(X_test_pooled_output.shape[1]==768)
    assert(len(y_test)==len(X_test_pooled_output))
    assert(len(y_train.shape)==1)
    assert(len(X_train_pooled_output.shape)==2)
    assert(len(y_test.shape)==1)
    assert(len(X_test_pooled_output.shape)==2)
    return True
greader_output()

True

<pre><font size=6>Part-5: Training a NN with 768 features</font>

Create a NN and train the NN. 
1.<b> You have to use AUC as metric.</b> 
2. You can use any architecture you want. 
3. You have to use tensorboard to log all your metrics and Losses. You have to send those logs. 
4. Print the loss and metric at every epoch. 
5. You have to submit without overfitting and underfitting. 
</pre>

In [ ]:
##imports
#from tensorflow.keras.layers import Input, Dense, Activation, Dropout,MaxPooling1D,Conv1D
#from tensorflow.keras.models import Model
#import datetime
#from sklearn.metrics import roc_auc_score


In [ ]:

def aucroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

In [ ]:
##create an NN and 

tf.keras.backend.clear_session()
## Set the random seed values to regenerate the model.
np.random.seed(0)
rn.seed(0)

#Input layer
input_layer1 = Input(shape=( 768 , ),name='Input_Layer')
#conv1d1 = Conv1D(filters=64, kernel_size=1 ,strides=(1), activation='relu')(input_layer1)
#max_pool_1 = MaxPooling1D(  pool_size=2, strides= 1 , padding="valid", data_format="channels_last", name="MaxPool1" )(input_layer1)
#dropout_1 = Dropout(0.2)(max_pool_1)
dense1 = Dense(units=64,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=29),name='dense1')(input_layer1)
dropout_2 = Dropout(0.1)(dense1)
#dense2 = Dense(units=256,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30),name='dense2')(dense1)
#dropout_2 = Dropout(0.5)(dense1)
#dense3 = Dense(units=128,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=31),name='dense3')(dense2)
#dropout_1 = Dropout(0.1)(dense3)

#dense4 = Dense(units=32,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=32),name='dense4')(dropout_1)
#dropout_2 = Dropout(0.1)(dense4)

#dense5 = Dense(units=16,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=33),name='dense5')(dropout_2)
#GlobalMaxPooling1D()
#dropout_3 = Dropout(0.1)(dense5)

out1 = Dense(units=1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=34),name='output')(dropout_2)

'''
model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length))
model.add(Conv1D(64, 8, activation = 'relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
'''
model_nn = Model(inputs= input_layer1 , outputs= out1 )

#model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy',aucroc])
model_nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss='binary_crossentropy',metrics=['accuracy', aucroc ])

model_nn.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (InputLayer)    [(None, 768)]             0         
                                                                 
 dense1 (Dense)              (None, 64)                49216     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 output (Dense)              (None, 1)                 65        
                                                                 
Total params: 49,281
Trainable params: 49,281
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_nn.fit(x = X_train_pooled_output , y=y_train , batch_size=256 , epochs=10 , \
          validation_data=(X_test_pooled_output , y_test ) , validation_batch_size=256  )


Epoch 1/10
313/313 [==============================] - 3s 10ms/step - loss: 0.2238 - accuracy: 0.9068 - aucroc: 0.9269 - val_loss: 0.2515 - val_accuracy: 0.9007 - val_aucroc: 0.9310
Epoch 2/10
313/313 [==============================] - 3s 9ms/step - loss: 0.2135 - accuracy: 0.9122 - aucroc: 0.9324 - val_loss: 0.2016 - val_accuracy: 0.9172 - val_aucroc: 0.9354
Epoch 3/10
313/313 [==============================] - 2s 6ms/step - loss: 0.2112 - accuracy: 0.9134 - aucroc: 0.9361 - val_loss: 0.2263 - val_accuracy: 0.9020 - val_aucroc: 0.9370
Epoch 4/10
313/313 [==============================] - 2s 6ms/step - loss: 0.2106 - accuracy: 0.9141 - aucroc: 0.9367 - val_loss: 0.2086 - val_accuracy: 0.9146 - val_aucroc: 0.9412
Epoch 5/10
313/313 [==============================] - 2s 6ms/step - loss: 0.2028 - accuracy: 0.9172 - aucroc: 0.9401 - val_loss: 0.2634 - val_accuracy: 0.8938 - val_aucroc: 0.9387
Epoch 6/10
313/313 [==============================] - 2s 7ms/step - loss: 0.2040 - accuracy: 0.9166

In [ ]:
model_nn.evaluate( X_test_pooled_output , y_test , batch_size=256)

79/79 [==============================] - 0s 4ms/step - loss: 0.1959 - accuracy: 0.9206 - aucroc: 0.9426


[0.19594620168209076, 0.9205999970436096, 0.9426466822624207]

In [ ]:
#log_dir= data_path + "logs/fit/" + datetime.datetime.now().strftime("%d%m%Y-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=0, write_graph=True,write_grads=True)

# old Model

model.fit(x = X_train_pooled_output , y=y_train , batch_size=256 , epochs=10 , \
          validation_data=(X_test_pooled_output , y_test ) , validation_batch_size=256  )



Epoch 1/5
  2/800 [..............................] - ETA: 29s - loss: 1.0252e-07 - accuracy: 0.1400 - aucroc: 0.6461WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0076s vs `on_train_batch_end` time: 0.0664s). Check your callbacks.


800/800 [==============================] - 4s 6ms/step - loss: 1.0370e-07 - accuracy: 0.1301 - aucroc: 0.5241 - val_loss: 1.0370e-07 - val_accuracy: 0.1301 - val_aucroc: 0.5229
Epoch 2/5
800/800 [==============================] - 5s 6ms/step - loss: 1.0370e-07 - accuracy: 0.1301 - aucroc: 0.5236 - val_loss: 1.0370e-07 - val_accuracy: 0.1301 - val_aucroc: 0.5229
Epoch 3/5
800/800 [==============================] - 4s 5ms/step - loss: 1.0370e-07 - accuracy: 0.1301 - aucroc: 0.5240 - val_loss: 1.0370e-07 - val_accuracy: 0.1301 - val_aucroc: 0.5229
Epoch 4/5
800/800 [==============================] - 4s 5ms/step - loss: 1.0370e-07 - accuracy: 0.1301 - aucroc: 0.5235 - val_loss: 1.0370e-07 - val_accuracy: 0.1301 - val_aucroc: 0.5229
Epoch 5/5
800/800 [==============================] - 4s 6ms/step - loss: 1.0370e-07 - accuracy: 0.1301 - aucroc: 0.5244 - val_loss: 1.0370e-07 - val_accuracy: 0.1301 - val_aucroc: 0.5229


In [ ]:
log_dir= data_path + "logs/fit/" + datetime.datetime.now().strftime("%d%m%Y-%H%M%S")
log_dir

<Pre><font size=6>Part-6: Creating a Data pipeline for BERT Model</font> 

1. load data
2. Read the csv file
3. Remove all the html tags
4. Now do tokenization [Part 3 as mentioned above]
    * Create tokens,mask array and segment array
5. Get Embeddings from BERT Model [Part 4 as mentioned above] , let it be X_test
   * Print the shape of output(X_test.shape).You should get (352,768)
6. Predit the output of X_test with the Neural network model which we trained earlier.
7. Print the occurences of class labels in the predicted output

</pre>

In [ ]:
test_data = pd.read_csv( '/content/26_NLP_Transfer/' +'test.csv')
#data_path
test_data.shape

(352, 1)

In [ ]:
test_data.head()

,Text
0,Just opened Greenies Joint Care (individually ...
1,This product rocks :) My mom was very happy w/...
2,"The product was fine, but the cost of shipping..."
3,I love this soup. It's great as part of a meal...
4,Getting ready to order again. These are great ...


In [ ]:
test_data['Text'] = test_data['Text'].apply( text_preprocess )

X_te_1 = test_data['Text']


In [ ]:
test_data.shape , X_te_1.shape #, X_te_1[0].shape

((352, 1), (352,))

In [ ]:
X_te_1[0:10].values

array(['just opened greenies joint care  individually sealed  in december 2011 and found small worm crawling all over it   next one looked fine  but really supposed to trust these now ',
       'this product rocks    my mom was very happy w the product it was excatly as described we loved seeing all the candy and eating it all   ',
       'the product was fine  but the cost of shipping was more than the cost of the tea   wo not make that mistake again ',
       'i love this soup  it is great as part of a meal or as a nutritious and satisifying low in calorie snack  for a light lunch  i stir in some shreeded cheese ',
       'getting ready to order again  these are great because they compost very quickly  but  some of the bags have tears in the bottom  so remember to always carry extra so you have emergency  back up  bags   a bit costly but  again  better for the environment ',
       'these were delicious  but not wrapped as well as i think they should be   they kind of melted out of t

In [ ]:
X_te_tokens, X_te_mask, X_te_segment = [] , [] , []

for i in range( X_te_1.shape[0]):
    
    tokens = tokenizer.tokenize( ( X_te_1[i] ) )
    tokens = tokens[ 0 : ( max_seq_length - 2 )]
    tokens = ['[CLS]',*tokens,'[SEP]']
    
    X_te_tokens.append( np.array( tokenizer.convert_tokens_to_ids(tokens) +  [0]*( max_seq_length - len(tokens))  ) )
    
    X_te_mask.append( np.array([1]*len(tokens) + [0]*( max_seq_length - len(tokens))))
    
    X_te_segment.append(np.array( [0]*( max_seq_length )))

X_te_tokens = np.array( X_te_tokens )
X_te_mask = np.array( X_te_mask )
X_te_segment = np.array( X_te_segment )

In [ ]:
X_te_tokens.shape , X_te_mask.shape , X_te_segment.shape

((352, 55), (352, 55), (352, 55))

In [ ]:
X_test_pooled_op = bert_model.predict([ X_te_tokens , X_te_mask , X_te_segment ]) 

In [ ]:
X_test_pooled_op.shape

(352, 768)

In [ ]:
predicted_y = model_nn.predict( X_test_pooled_op )

In [ ]:
print(predicted_y.shape)
predicted_y

(352, 1)


array([[0.64654607],
       [0.99570906],
       [0.36893794],
       [0.9600499 ],
       [0.9803402 ],
       [0.78969705],
       [0.8268261 ],
       [0.994932  ],
       [0.994044  ],
       [0.9935108 ],
       [0.9024153 ],
       [0.9556741 ],
       [0.554749  ],
       [0.9157876 ],
       [0.92947936],
       [0.43624252],
       [0.99680823],
       [0.9114974 ],
       [0.87107605],
       [0.955121  ],
       [0.724998  ],
       [0.97300977],
       [0.7591065 ],
       [0.92691225],
       [0.95712703],
       [0.9699526 ],
       [0.9803175 ],
       [0.9193082 ],
       [0.9291376 ],
       [0.9898239 ],
       [0.53297853],
       [0.92045456],
       [0.95925915],
       [0.98949885],
       [0.9711936 ],
       [0.9971016 ],
       [0.51176274],
       [0.9905184 ],
       [0.99745697],
       [0.99220204],
       [0.32184538],
       [0.99750465],
       [0.9708457 ],
       [0.9354391 ],
       [0.31295896],
       [0.7705564 ],
       [0.9873089 ],
       [0.800

In [ ]:
predicted_label = [1 if i > 0.5 else 0 for i in predicted_y ]

labley , county = np.unique( np.array( predicted_label ),  return_counts=True)

dict(zip(labley , county ))

{0: 58, 1: 294}